In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import random
import time
import pandas as pd
from tqdm import tqdm

In [ ]:
def get_driver(username = "", password = ""):
    
    driver = webdriver.Firefox(executable_path='geckodriver')#, options=options)
    url = "http://pss-system.cnipa.gov.cn/sipopublicsearch/portal/uilogin-forwardLogin.shtml"
    driver.get(url)
    time.sleep(1)
    
    driver.find_element_by_id("j_username").send_keys(username)
    driver.find_element_by_id("j_password_show").send_keys(password)
    valid = input("驗證碼：")
    driver.find_element_by_id("j_validation_code").send_keys(valid)
    driver.find_element_by_link_text("登录").click()
    
    return driver

In [ ]:
def get_table(driver, ID, GrantNo):
    
    delay = 7
    mini_delay = 1.5
    try:
        myElem = WebDriverWait(driver, delay).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="queryQuoteb"]')))
        
        myElem = WebDriverWait(driver, delay).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="queryQuoteb"]')))
        
    except TimeoutException:
                print(f"Loading search button took too much time! (for ID {ID})")
    
    time.sleep(mini_delay)
    
    driver.find_element_by_id("quoteValue").clear()
    driver.find_element_by_id("quoteValue").send_keys(GrantNo)
    driver.find_element_by_xpath('//*[@id="queryQuoteb"]').click()
    
    something_XPATH = "/html/body/div[3]/div[2]/div/div[1]/div[2]/table/tbody/tr[1]/td[3]"
    nothing_XPATH = "/html/body/div[3]/div[2]/div/div[1]/div[2]/table/tbody/tr/td"
    try:
        WebDriverWait(driver, delay).until(
            lambda driver: driver.find_elements(By.XPATH, something_XPATH) or driver.find_elements(By.XPATH, nothing_XPATH))
    except:
        return None, False
    
    total_page_length = 1
    try:
        html_list = driver.find_element_by_class_name("m-pagination-page")
        total_page_length = len(html_list.find_elements_by_tag_name("li"))
    except:
        pass
    
    ID_list = []
    GrantNo_list = []
    相关性 = []
    公开号 = []
    IPC分类号 = []
    相关段落 = []
    相关权利要求 = []
    prefix = '/html/body/div[3]/div[2]/div/div[1]/div[2]/table/tbody'
    for i in range(1, total_page_length + 1):

        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, something_XPATH)))
        #print("now step", i)
        if(i != total_page_length):
            try:
                myElem = WebDriverWait(driver, delay).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "m-pagination-page")))
                
                myElem = WebDriverWait(driver, delay).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, "m-pagination-page")))
                
            except TimeoutException:
                print(f"Loading button took too much time! (for ID {i}-1)")
                
        try:
            html_list = driver.find_element_by_class_name("m-pagination-page")
            page_items = html_list.find_elements_by_tag_name("li")
        except:
            pass
        
        table_length = len(driver.find_elements_by_xpath(f'{prefix}/tr'))

        for row in range(1, table_length+1):
            ID_list.append(ID)
            GrantNo_list.append(GrantNo)
            相关性.append(driver.find_element_by_xpath(f'{prefix}/tr[{row}]/td[{2}]').text)
            公开号.append(driver.find_element_by_xpath(f'{prefix}/tr[{row}]/td[{3}]').text)
            IPC分类号.append(driver.find_element_by_xpath(f'{prefix}/tr[{row}]/td[{4}]').text)
            相关段落.append(driver.find_element_by_xpath(f'{prefix}/tr[{row}]/td[{5}]').text)
            相关权利要求.append(driver.find_element_by_xpath(f'{prefix}/tr[{row}]/td[{6}]').text)

        if(i != total_page_length):
            try:
                myElem = WebDriverWait(driver, delay).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "m-pagination-page")))
                
                myElem = WebDriverWait(driver, delay).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, "m-pagination-page")))
                
            except TimeoutException:
                print(f"Loading button took too much time! (for ID {i}-2)")
            
            time.sleep(mini_delay)
            
            html_list = driver.find_element_by_class_name("m-pagination-page")
            page_items = html_list.find_elements_by_tag_name("li")
            page_items[i].click()

    final = pd.DataFrame({"id":ID_list,
                          "GrantNo":GrantNo_list,
                          "相关性" : 相关性,
                          "公开号":公开号,
                          "IPC分类号":IPC分类号,
                          "相关段落":相关段落,
                          "相关权利要求":相关权利要求})

    #print(final)
    return final, True

In [ ]:
#driver = get_driver(username = "", password = "")

In [ ]:
#get_table(driver, 0, "CN102340805B")

In [ ]:
from IPython.display import clear_output

driver = get_driver(username = "", password = "")
grant_No_csv = pd.read_excel("citation_data.xlsx", sheet_name='資料')

while(1):
    clear_output(wait=True)
    driver.get("http://pss-system.cnipa.gov.cn/sipopublicsearch/search-ui/app/searchtools/quote.jsp")
    
    try:
        myElem = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID, 'queryQuoteb')))
        print("Login completed, page is ready!")
    except TimeoutException:
        print("Loading took too much time!")

    try:
        start_ID = 0
        original_table = pd.DataFrame()
        try:
            original_table = pd.read_csv("citation_output.csv")
            start_ID = int(original_table['id'].max())
        except:
            pass

        print(f"Started from ID {start_ID}")

        for i in tqdm(range(start_ID, grant_No_csv.shape[0])):

            ID = grant_No_csv.iloc[i][0]
            GrantNo = grant_No_csv.iloc[i][1]

            result_table, if_get_table = get_table(driver, ID, GrantNo)

            if not if_get_table:
                continue

            original_table = pd.concat([original_table, result_table])
            original_table.to_csv("citation_output.csv", index = 0, encoding='utf_8_sig')

            if i % 3 == 0:
                time.sleep(3 + random.random() * 7)

    except Exception as e:
        print(e)